### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** 25 ноября 2019, 15:00   
**Штраф за опоздание:** -2 балла после 15:00 25 ноября, -4 балла после 15:00 2 декабря, -6 баллов после 15:00 9 декабря  -8 баллов после 15:00 16 декабря.

При отправлении ДЗ указывайте фамилию в названии файла Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:
[ML0919, Задание 3] Фамилия Имя. 


Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (3 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на [wine](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html) и [Speed Dating Data](https://cloud.mail.ru/public/8nHV/p6J7wY1y1)

###### Задание 2 (3 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine и Speed Dating Data. 
Для этого используем numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw3.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic

In [15]:
a = np.array([1, 2, 3, 2, 3, 4])
e = np.bincount(a, minlength=10)
print(e)
e[:-1]

[0 1 2 2 1 0 0 0 0 0]


array([0, 1, 2, 2, 1, 0, 0, 0, 0])

In [228]:
#%%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return l_s + r_s  - (np.sum(l_c ** 2 / l_s, axis=1, keepdims=True) + \
                    np.sum(r_c ** 2 / r_s, axis=1, keepdims=True))
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        return -(np.sum(l_c/l_s*np.log2(l_c/l_s), axis=1, keepdims=True) + \
                np.sum(r_c/r_s * np.log2(r_c / r_s), axis=1, keepdims=True))

    def __misclass(self, l_c, l_s, r_c, r_s):
        return l_s + r_s - (np.max(l_c, axis=1, keepdims=True) + \
                             + np.max(r_c, axis=1, keepdims=True))
    
    def __gini_gain(self, y):
        _, counts = np.unique(y, return_counts=True)
        return (1 - np.sum(counts ** 2 / y.size ** 2))

    def __entropy_gain(self, y):
        _, counts = np.unique(y, return_counts=True)
        return -(np.sum((counts / y.size) * \
                                  np.log2(counts / y.size)))
    def __misclass_gain(self, y):
        _, counts = np.unique(y, return_counts=True)
        return (1 - np.max(counts / y.size))
    
    def __importance (self, y, y_l, y_r, index):
        if self.G_function == self.__gini:
            func = self.__gini_gain
        elif self.G_function == self.__entropy:
            func = self.__entropy_gain
        elif self.G_function == self.__misclass:
            func = self.__misclass_gain
        self.feature_importances_[index] += (y.size*func(y) - \
                                             y_l.size*func(y_l) - y_r.size*func(y_r))

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.sqrt(n_feature))] 
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.log2(n_feature))]

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = self.num_class
        cut_size = np.int(self.min_samples_split / 2 - 1)
        if cut_size == 0:
            splited_sorted_y = sorted_y
        else:
            splited_sorted_y = sorted_y[cut_size : -cut_size]
        r_border_ids = np.where(splited_sorted_y[:-1] !=
                                splited_sorted_y[1:])[0] + (cut_size +1)
            
        if len(r_border_ids) == 0:
            return np.inf, None
        
        eq_el_count = r_border_ids - np.append(np.array([cut_size]),
                                              r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]),
                    sorted_y[r_border_ids - 1]] = 1
        
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + \
            np.bincount(sorted_y[:cut_size], minlength=class_number)
        
        l_class_count = np.cumsum(class_increments, axis=0)
        r_class_count = np.bincount(sorted_y,
                                    minlength=class_number) - l_class_count
        l_size = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_size = sorted_y.shape[0] - l_size
        
        gs = self.G_function(l_class_count, l_size, r_class_count, r_size)
        idx = np.argmin(gs)
        left_el_id = l_size[idx][0]
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2

    def __fit_node(self, x, y, node_id, depth):
        cnt = np.bincount(y, minlength=self.num_class) # какой из игрек встречается чаще
        if ((self.max_depth is not None and depth == self.max_depth) or
           (self.min_samples_split is not None and \
            x.shape[0] < self.min_samples_split)):
                self.tree[node_id] = (self.LEAF_TYPE, np.argmax(cnt),
                                     cnt / cnt.size)
                return 
        
        gs = np.zeros(x.shape[1])
        thresholds = np.zeros(x.shape[1])
        idx = self.get_feature_ids(x.shape[1])
        for i in idx:
            gs[i], thresholds[i] = self.__find_threshold(x[:, i], y)
            
        index = np.argmin(gs)
        
        if np.isnan(thresholds[index]):
            self.tree[node_id] = (self.LEAF_TYPE, np.argmax(cnt),
            cnt / cnt.size)
            return
        
        x_l, x_r, y_l, y_r = self.__div_samples(x, y, index, thresholds[index])
        
        if y_l.shape[0] == 0 or y_r.shape[0] == 0:
            self.tree[node_id] = (self.LEAF_TYPE, np.argmax(cnt),
                                  cnt / cnt.size)
            return
            
        self.tree[node_id] = (self.NON_LEAF_TYPE, index, thresholds[index])
        
        self.__importance(y, y_l, y_r, index)
        self.__fit_node(x_l, y_l, 2 * node_id + 1, depth + 1)
        self.__fit_node(x_r, y_r, 2 * node_id + 2, depth + 1)
    
    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.feature_importances_ = np.zeros(x.shape[1])
        self.__fit_node(x, y, 0, 0) 
        self.feature_importances_ /= y.size

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [229]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [230]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

In [231]:
X_train.shape

(160, 13)

## Проверка скорости работы на wine

In [232]:
%time clf.fit(X_train, y_train)

CPU times: user 1.63 ms, sys: 876 µs, total: 2.5 ms
Wall time: 1.19 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [233]:
%time my_clf.fit(X_train, y_train)

CPU times: user 18.4 ms, sys: 1.85 ms, total: 20.2 ms
Wall time: 22.9 ms


## Проверка качества работы на wine

In [234]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.8888888888888888

In [235]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.8888888888888888

## Подготовка данных Speed Dating Data 

In [253]:
df = pd.read_csv('Speed Dating Data.csv',
                 encoding='cp1251')

df = df.iloc[:, :97]
df = df.drop(['idg', 'id','condtn', 'age_o', 'round', 'race_o', 'pf_o_att',
              'pf_o_sin','position', 'positin1', 'pf_o_int',
              'pf_o_fun', 'pf_o_amb', 'pf_o_sha',
              'dec_o', 'attr_o', 'order', 'sinc_o', 'partner','intel_o', 'fun_o',
              'amb_o', 'shar_o', 'like_o', 'prob_o', 'met_o'],
             axis=1)

df = df.dropna(subset=['age'])

df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(0)
df = df.drop(['field'], axis=1)
pd.get_dummies(df, columns=['field_cd'],
               prefix='field_cd', prefix_sep='=')
df = df.drop(['undergra'], axis=1)
df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)
df.loc[:, 'mn_sat'] = df.mn_sat.fillna(-999)
df.loc[:, 'tuition'] = df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)
df.loc[:, 'tuition'] = df.tuition.fillna(-999)
df = df.dropna(subset=['imprelig', 'imprace'])
df = df.drop(['from', 'zipcode'], axis=1)
df.loc[:, 'income'] = df.loc[:, 'income'].str.replace(',', '').astype(np.float)
df.loc[:, 'income'] = df.loc[:, 'income'].fillna(-999)
df = df.dropna(subset=['date'])
df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(0)
df = df.drop(['career'], axis=1)
df = df.drop(['sports', 'tvsports', 'exercise', 'dining', 'museums', 'art', 'hiking', 'gaming',
              'clubbing', 'reading', 'tv', 'theater', 'movies', 'concerts', 'music', 'shopping', 'yoga'], axis=1)
df = df.drop(['expnum'], axis=1)
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']].sum(axis=1)
df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']] = \
    (df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']].T / df.loc[:,
                                                                                      'temp_totalsum'].T).T * 100
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']].sum(axis=1)
df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']] = \
    (df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']].T / df.loc[:,
                                                                                      'temp_totalsum'].T).T * 100
df = df.drop(['temp_totalsum'], axis=1)
for i in [4, 5]:
    feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i),
            'intel{}_1'.format(i), 'fun{}_1'.format(i),
            'amb{}_1'.format(i), 'shar{}_1'.format(i)]

    if i != 4:
        feat.remove('shar{}_1'.format(i))

    df = df.drop(feat, axis=1)
df = df.drop(['wave'], axis=1)
df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid']) \
    .drop(['gender'], axis=1).dropna()
df_female = df.query('gender == 0').drop_duplicates(subset=['iid']) \
    .drop(['gender', 'match', 'int_corr', 'samerace'], axis=1).dropna()

df_female.columns = df_female.columns + '_f'
df_female = df_female.drop(['pid_f'], axis=1)
df_pair = df_male.join(df_female.set_index('iid_f'),
                       on='pid',
                       how='inner')
df_pair = df_pair.drop(['iid', 'pid'], axis=1)
print(df_pair.columns)
X = df_pair.iloc[:, 1:].values
y = df_pair.iloc[:, 0].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)


Index(['match', 'int_corr', 'samerace', 'age', 'field_cd', 'mn_sat', 'tuition',
       'race', 'imprace', 'imprelig', 'income', 'goal', 'date', 'go_out',
       'career_c', 'exphappy', 'attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1',
       'amb1_1', 'shar1_1', 'attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1',
       'amb2_1', 'shar2_1', 'attr3_1', 'sinc3_1', 'fun3_1', 'intel3_1',
       'amb3_1', 'age_f', 'field_cd_f', 'mn_sat_f', 'tuition_f', 'race_f',
       'imprace_f', 'imprelig_f', 'income_f', 'goal_f', 'date_f', 'go_out_f',
       'career_c_f', 'exphappy_f', 'attr1_1_f', 'sinc1_1_f', 'intel1_1_f',
       'fun1_1_f', 'amb1_1_f', 'shar1_1_f', 'attr2_1_f', 'sinc2_1_f',
       'intel2_1_f', 'fun2_1_f', 'amb2_1_f', 'shar2_1_f', 'attr3_1_f',
       'sinc3_1_f', 'fun3_1_f', 'intel3_1_f', 'amb3_1_f'],
      dtype='object')


## Проверка скорости работы на Speed Dating Data 

In [254]:
# тут должен быть код типа f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')
%time clf.fit(X_train, y_train)

CPU times: user 67.8 ms, sys: 1.54 ms, total: 69.4 ms
Wall time: 67.9 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [255]:
# тут должен быть код типа f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')
%time my_clf.fit(X_train, y_train)

CPU times: user 560 ms, sys: 5.98 ms, total: 566 ms
Wall time: 565 ms


## Проверка качества работы на Speed Dating Data

In [256]:
# тут должен быть код типа %time clf.fit(X_train, y_train)
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.5767195767195767

In [257]:
# тут должен быть код типа %time my_clf.fit(X_train, y_train)
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.5402385483597445

## Задание 3

In [261]:
top10_features = np.argsort(my_clf.feature_importances_)[::-1][:10]
df = df_pair.columns[1:]
df[top10_features]

Index(['date', 'int_corr', 'age', 'fun2_1_f', 'race', 'amb1_1_f', 'imprace',
       'income_f', 'imprelig', 'attr3_1'],
      dtype='object')

In [262]:
top10_features = np.argsort(clf.feature_importances_)[::-1][:10]
df = df_pair.columns[1:]
df[top10_features]

Index(['int_corr', 'income_f', 'field_cd_f', 'fun2_1_f', 'race_f',
       'exphappy_f', 'sinc1_1', 'intel1_1', 'intel2_1_f', 'age_f'],
      dtype='object')

## Задание 4

In [264]:
param = {"max_depth": range(2, 10),
         "min_samples_leaf" : range(2, 10),
         "max_features": ['sqrt', 'log2'],
         "criterion": ["gini", "entropy"],
}
forest = RandomizedSearchCV(RandomForestClassifier(random_state = 123), param, n_iter= 50)
forest.fit(X_train, y_train)
forest.best_params_

/Users/sonyaserebryakova/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/sonyaserebryakova/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/sonyaserebryakova/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/sonyaserebryakova/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10

/Users/sonyaserebryakova/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/sonyaserebryakova/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/sonyaserebryakova/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/sonyaserebryakova/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarni

/Users/sonyaserebryakova/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/sonyaserebryakova/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/sonyaserebryakova/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/sonyaserebryakova/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarni

/Users/sonyaserebryakova/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/sonyaserebryakova/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/sonyaserebryakova/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/sonyaserebryakova/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarni

/Users/sonyaserebryakova/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/sonyaserebryakova/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/sonyaserebryakova/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


{'min_samples_leaf': 2,
 'max_features': 'sqrt',
 'max_depth': 8,
 'criterion': 'gini'}

## Задание 5